In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from mosaicml import *
from mosaicml.constants import MLModelFlavours
from werkzeug.datastructures import FileStorage
import pandas as pd
import os, tempfile

# import time
# time.sleep(1000)
print("-----creating spark session")
spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
spark.sparkContext.setLogLevel('WARN')

print("-----reading data from azure")
storage_account_name = "pocmosaic"
storage_account_access_key = "iX6jZejeNh1bUejdSRYk+4X2S/pAbTy+idkTM5UAw2HxgxSnaYGJSN+ffzZAQ5/Q6xQe2Ja74Re9nKHMVmsGhA=="
file_location = "wasbs://container1@pocmosaic.blob.core.windows.net/catalog/local_file_upload/RatanBBostonHousing.csv"
file_type = "csv"
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)
dataset = spark.read.option("header", "true").format(file_type).option("inferSchema", "true").load(file_location)

print("-----splitting data")
train_data,test_data = dataset.randomSplit([0.8,0.2])

print("-----creating pipeline")
stage_1 = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
# define stage 5: logistic regression model                          
stage_2 = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

# setup the pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2])
print("-----training pipeline")
regression_pipeline = regression_pipeline.fit(train_data)
test_update = regression_pipeline.transform(test_data)
# test_update.show()

#Predict using the model
print("-----test data")
test_update.show()

# define score function
def score(model, request):
    data = request.json
    df = pd.DataFrame(data, columns=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv'])
    spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
    spark_df = spark.createDataFrame(df)
    return model.transform(spark_df).toPandas()["prediction"].to_json()

print("-----defining & testing score function")
import requests
req = requests.Request()
req.json = [[1, -0.1, 0.0425, 0.0279, 2.4716, -15.2808, 4.2783, -0.0005, -1.4009, 0.2692, -0.013, -0.9905, 0.0084, -0.5533]]
rez = score(regression_pipeline, req)
print("the prediction is {}".format(rez))

print("-----register model")
register_model(regression_pipeline, score, "Boston_Regression_Analysis_jun30", "pyspark model", MLModelFlavours.pyspark)